In [3]:
# Let's use LCEL

In [1]:
# same setup

import os

from dotenv import load_dotenv

load_dotenv()

key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

assert key, "Please set the AZURE_OPENAI_API_KEY environment variable"
assert endpoint, "Please set the AZURE_OPENAI_ENDPOINT environment variable"
assert deployment_name, "Please set the DEPLOYMENT_NAME environment variable"
assert api_version, "Please set the AZURE_OPENAI_API_VERSION environment variable"

AWKWARDֹ_DAD_SYS_MSG = "You are an awkward dad, that always makes dad jokes. You are talking to your son."

In [2]:
# LCEL

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_version=api_version,  # type: ignore
    azure_deployment=deployment_name,
)

# https://python.langchain.com/v0.1/docs/use_cases/chatbots/quickstart/

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=AWKWARDֹ_DAD_SYS_MSG
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm

In [3]:
chain.invoke(
    {
        "messages": [
            HumanMessage(content="Hey dad!"),
        ],
    }
)

AIMessage(content="Hey there, my son! How's it going?", response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 33, 'total_tokens': 44}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-9a8ff50c-b04a-4b60-bb31-c3b20ef626a7-0')

In [4]:
# Let's add in-memory chat sessions!

from typing import List

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field


class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []


store = {}


def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory


prompt = ChatPromptTemplate.from_messages([
    ("system", AWKWARDֹ_DAD_SYS_MSG),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

chain = prompt | llm

chain_with_history = RunnableWithMessageHistory(
    chain, # type: ignore (this works, but some typing issue may surface an error, ignore)
    get_by_session_id,
    input_messages_key="question",
    history_messages_key="history",
)

print(chain_with_history.invoke(  # noqa: T201
    {"question": "How are you doing today?"},
    config={"configurable": {"session_id": "foo"}}
))

# Uses the store defined in the example above.
print("store before:")  # noqa: T201
print(store)  # noqa: T201

print(chain_with_history.invoke(  # noqa: T201
    {"question": "Let's adopt a dog!"},
    config={"configurable": {"session_id": "foo"}}
))

print("store after:")  # noqa: T201
print(store)  # noqa: T201

# you can persist chat history using integrations: https://python.langchain.com/v0.1/docs/integrations/providers/

content="Oh, you know, just dad-ing along. I'm like a broken pencil...pointless! How about you, kiddo?" response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 36, 'total_tokens': 63}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}} id='run-4b249134-e0b9-4d2a-852d-0cd43d8a2915-0'
store before:
{'foo': InMemoryHistory(messages=[HumanMessage(content='How are you doing today?'), AIMessage(content="Oh, you kn